In [6]:
import numpy as np
import pathlib
import importlib

RECON_PATH = pathlib.Path("../data/20241011_phantom_mono/run2/prostate/reconstruction/droid")


In [26]:
disps = np.load((RECON_PATH/"disps.npy").resolve())
images = np.load((RECON_PATH/"images.npy").resolve())
intrinsics = np.load((RECON_PATH/"intrinsics.npy").resolve())
poses = np.load((RECON_PATH/"poses.npy").resolve())
tstamps = np.load((RECON_PATH/"tstamps.npy").resolve())


In [27]:
disps.shape

(190, 328, 584)

In [28]:
disps[-1, :, :]

array([[2.787258 , 2.7830036, 2.7832267, ..., 3.7577796, 3.7594903,
        3.7528381],
       [2.7975404, 2.7931035, 2.792388 , ..., 3.7634213, 3.7655783,
        3.757675 ],
       [2.8029273, 2.7985337, 2.796724 , ..., 3.7629895, 3.761981 ,
        3.7545822],
       ...,
       [4.082106 , 4.091421 , 4.0903263, ..., 3.2092373, 3.21198  ,
        3.2138329],
       [4.0988607, 4.1106496, 4.110519 , ..., 3.2169824, 3.2184887,
        3.220656 ],
       [4.1113744, 4.125168 , 4.1311097, ..., 3.220003 , 3.2219715,
        3.224883 ]], dtype=float32)